In [ ]:
!nvidia-smi

Thu Aug  5 19:19:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    32W /  70W |   2146MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
uploaded = files.upload()

!rm -rf /root/.ssh
!mkdir /root/.ssh
# !tar -xvzf ssh.tar.gz -C /root/.ssh && rm -rf ssh.tar.gz && chmod 700 /root/.ssh
!tar -xvzf ssh.tar.gz -C /root/.ssh && rm -rf ssh.tar.gz
!ssh-keyscan -t rsa github.com >> /root/.ssh/known_hosts

!git config --global user.email 'ixomaxip@gmail.com'
!git config --global user.user 'ixomaxip'
!ssh -T git@github.com

!git clone git@github.com:ixomaxip/nasoc.git

Saving ssh.tar.gz to ssh.tar.gz
./id_rsa
./id_rsa.pub
# github.com:22 SSH-2.0-babeld-968490c5
Hi ixomaxip! You've successfully authenticated, but GitHub does not provide shell access.
Cloning into 'nasoc'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 64 (delta 24), reused 12 (delta 0), pack-reused 0
Receiving objects: 100% (64/64), 408.68 KiB | 2.13 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

In [ ]:
FULL_PATH = '/content/nasoc/data/en_Hasoc2021_train.csv'
TRAIN_PATH = '/content/nasoc/data/train.csv'
TEST_PATH = '/content/nasoc/data/validate.csv'

full_df = pd.read_csv(FULL_PATH)
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

print(f'''shapes:
  full: {full_df.shape}
  train: {train_df.shape}
  test: {test_df.shape}
''')
print(f'''classes:
  task_1: {full_df.task_1.unique()}
  task_2: {full_df.task_2.unique()}
''')
train_df.columns

shapes:
  full: (3843, 5)
  train: (3074, 7)
  test: (769, 7)

classes:
  task_1: ['HOF' 'NOT']
  task_2: ['PRFN' 'OFFN' 'NONE' 'HATE']



Index(['Unnamed: 0', '_id', 'text', 'task_1', 'task_2', 'inputs',
       'task_2_cat'],
      dtype='object')

In [ ]:
def prepare_df(path, task):
  df = pd.read_csv(path)
  df.rename(columns={f'task_{task}': 'y'}, inplace=True)
  df = df[['text', 'y']]
  return df

# spark-nlp FAILED

In [ ]:
!wget https://setup.johnsnowlabs.com/nlu/colab.sh -O - | bash 
import nlu
# import nlu-gpu

In [ ]:
train_t1 = prepare_df(TRAIN_PATH, task=1)
test_t1 = prepare_df(TEST_PATH, task=1)
train_t1.y.unique()

NameError: ignored

In [ ]:
trainable_pipe = nlu.load('xx.embed_sentence.labse train.sentiment')
trainable_pipe['sentiment_dl'].setMaxEpochs(60)  
trainable_pipe['sentiment_dl'].setLr(0.005)
fitted_pipe = trainable_pipe.fit(train_t1)
preds = fitted_pipe.predict(train_t1, output_level='document')
preds.dropna(inplace=True)

print(classification_report(preds['y'], preds['trained_sentiment']))
MODEL_PATH = './drive/MyDrive/nasoc/task1/sentiment_dl_baseline_trained'
fitted_pipe.save(MODEL_PATH)
preds

Mounted at /content/drive


In [ ]:
preds = fitted_pipe.predict(test_t1, output_level='document')
preds.dropna(inplace=True)
print(classification_report(preds['y'], preds['trained_sentiment']))

# LaBSE

In [ ]:
!pip install transformers
import torch
from transformers import AutoTokenizer, AutoModel

DEVICE = torch.device('cuda:0')
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")
model.eval().to(DEVICE)

In [ ]:
def text_encoder(texts, model, tokenizer, max_len=128):
  encoded_input = tokenizer(texts, padding=True, truncation=True, max_length=max_len, return_tensors='pt').to(device=DEVICE)
  with torch.no_grad():
    model_output = model(**encoded_input)
  embeddings = model_output.pooler_output
  embeddings = torch.nn.functional.normalize(embeddings)

  return embeddings

In [ ]:
!ls /content/drive/MyDrive/nasoc/

In [ ]:
ROOT_DIR = '/content/drive/MyDrive/nasoc/'
TRAIN_EMB = os.path.join(ROOT_DIR, 'train_emb.feather')
TEST_EMB = os.path.join(ROOT_DIR, 'test_emb.feather')

## Get labse embeddings

In [ ]:
# train_df['labse_emb'] = train_df.apply(lambda row: text_encoder(row['text'], model, tokenizer, 64)[0].detach().cpu().numpy(), axis=1)
# train_df.to_feather(TRAIN_EMB)

# test_df['labse_emb'] = test_df.apply(lambda row: text_encoder(row['text'], model, tokenizer, 64)[0].detach().cpu().numpy(), axis=1)
# test_df.to_feather(TEST_EMB)
train_df = pd.read_feather(TRAIN_EMB)
test_df = pd.read_feather(TEST_EMB)

In [ ]:
test_df.head(2)

,Unnamed: 0,_id,text,task_1,task_2,inputs,task_2_cat,labse_emb
0,4745,60c5d6bf5659ea5e55def8b8,4lakh cases in a day ðŸ˜” @PMOIndia @drharshva...,HOF,OFFN,0,2,"[-0.02545384, 0.03337645, 0.020745805, -0.0581..."
1,4795,60c5d6bf5659ea5e55def871,@SiennaSummerEIC @Chantel_Etoile You do unders...,HOF,PRFN,0,3,"[0.027386742, -0.0026722655, 0.0048503056, -0...."


## XGBOOST

In [ ]:
import xgboost as xgb

In [ ]:
x_train = np.stack(train_df.labse_emb.to_numpy())
y_train = train_df.inputs.values
print(x_train.shape)

x_test = np.stack(test_df.labse_emb.to_numpy())
y_test = test_df.inputs.values

(3074, 768)


In [ ]:
bst = xgb.XGBClassifier(
    objective='reg:logistic'
)

bst.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='reg:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_pred = bst.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.89      0.84       529
           1       0.66      0.47      0.55       240

    accuracy                           0.76       769
   macro avg       0.73      0.68      0.69       769
weighted avg       0.75      0.76      0.75       769

